In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [2]:
I = np.array([[1,0],[0,1]])
sx = np.array([[0,1],[1,0]])
sy = np.array([[0,-1j],[1j,0]])
sz = np.array([[1,0],[0,-1]])

sx2 = np.kron(np.kron(I,sx),I)
sz2 = np.kron(np.kron(I,sz),I)

sx_ab = np.kron(np.kron(sx,I),sx)
sy_ab = np.kron(np.kron(sy,I),sy)
sz_ab = np.kron(np.kron(sz,I),sz)

sx_ac = np.kron(np.kron(sx,sx),I)
sy_ac = np.kron(np.kron(sy,sy),I)
sz_ac = np.kron(np.kron(sz,sz),I)

sx_bc = np.kron(np.kron(I,sx),sx)
sy_bc = np.kron(np.kron(I,sy),sy)
sz_bc = np.kron(np.kron(I,sz),sz)

## Basis

In [3]:
#Computational basis
def sigmaz_basis(size,idx):   #size = number of particles
    basis = np.zeros(2**size).reshape(-1,1)
    basis[idx] = 1
    return basis

#Non hermitian spin(B) basis
b0 = np.array([1,1+1j]).reshape(2,1)
b1 = np.array([1-1j,1]).reshape(2,1)
b0_tilde = np.array([-1,1+1j]).reshape(2,1)
b1_tilde = np.array([1-1j,-1]).reshape(2,1)
ket_b = [b0,b1]
bra_b = [np.conj(b0_tilde).T,np.conj(b1_tilde).T]

In [4]:
#Biorthogonal Basis for system
def biorthogonal_basis(size,idx1,idx2):  #size = number of particles
    ket_basis = []
    bra_basis = []
    for i in range(2**idx1):
        for j in range(2**idx2):
            for k in range(2):
                basis1 = np.kron(sigmaz_basis(idx1,i),np.kron(ket_b[k],sigmaz_basis(idx2,j)))
                basis2 = np.kron(np.conj(sigmaz_basis(idx1,i)).T,np.kron(bra_b[k],np.conj(sigmaz_basis(idx2,j)).T))
                ket_basis.append(basis1)
                bra_basis.append(basis2)
    return ket_basis,bra_basis

## Hamiltonian

In [5]:
def I(idx):
    I = np.array([[1,0],[0,1]]).reshape(2,2)
    I_matrix = 1
    while idx>0:
        I_matrix = np.kron(I,I_matrix)
        idx-=1
    return I_matrix

In [6]:
def Hamiltonian(size,idx,hx,hz,d):   #size = number of particles ; idx = position of non-hermitian spin
    H = []
    sx = np.array([[0,1],[1,0]]).reshape(2,2)
    sy = np.array([[0,-1j],[1j,0]]).reshape(2,2)
    sz = np.array([[1,0],[0,-1]]).reshape(2,2)
    sxsx = np.kron(sx,sx)
    sysy = np.kron(sy,sy)
    szsz = np.kron(sz,sz)
    SXSX = []
    SYSY = []
    SZSZ = []
    for i in range(size-1):
        SX = np.kron(I(i),np.kron(sxsx,I(size-i-2)))
        SY = np.kron(I(i),np.kron(sysy,I(size-i-2)))
        SZ = np.kron(I(i),np.kron(szsz,I(size-i-2)))
        if (size % 2) == 0:                                                #For even size
            if(idx > size/2):
                SZ_nonherm = np.kron(I(size/2),np.kron(sz,I((size/2)-1)))
                SX_nonherm = np.kron(I(size/2),np.kron(sx,I((size/2)-1)))
            else:
                SZ_nonherm = np.kron(I((size/2)-1),np.kron(sz,I(size/2)))
                SX_nonherm = np.kron(I((size/2)-1),np.kron(sx,I(size/2)))
        else:
            SZ_nonherm = np.kron(I((size-1)/2),np.kron(sz,I((size-1)/2)))  #For odd size only
            SX_nonherm = np.kron(I((size-1)/2),np.kron(sx,I((size-1)/2)))  #For odd size only
        SXSX.append(SX)
        SYSY.append(SY)
        SZSZ.append(SZ)
        i+=1
# Interaction terms in hamiltonian
    
    for k in range(len(hz)): 
        H_int = 0
        for i in range(len(SXSX)):
            H_int = H_int + SXSX[i] + SYSY[i] + d[k]*SZSZ[i]
    
# Non-hermitian term of hamiltonian
        H_nonherm = 1j*hz[k]*SZ_nonherm + hx[k]*SX_nonherm
    
#Periodic boundary term of hamiltonian
        SX_pb = np.kron(sx,np.kron(I(size-2),sx))
        SY_pb = np.kron(sy,np.kron(I(size-2),sy))
        SZ_pb = np.kron(sz,np.kron(I(size-2),sz))
        H_pb = SX_pb + SY_pb + d[k]*SZ_pb
    
#Total hamiltonian    
        H_t = H_int + H_nonherm + H_pb
        H.append(H_t)
    return(H)

In [7]:
def eigvectors(H):
    H_dag = np.conj(H).T
    e,psi = np.linalg.eig(H)
    f,phi = np.linalg.eig(H_dag)
    ket_psi = psi[:,np.argmin(e)].reshape(-1,1)
    ket_phi = phi[:,np.argmin(f)].reshape(-1,1)
    return ket_psi, ket_phi

## Biorthogonal rdm

In [8]:
def bi_rho_c(dm,idx1,idx2):
    p,q,r,s = 0,0,0,0
    for i in range(2**idx1):
        for j in range(2**idx2):
            p = p + 0.5*(np.kron(np.conj(sigmaz_basis(idx1,i)).T,np.kron(bra_b[0],np.conj(sigmaz_basis(idx2,j)).T)) @ dm 
                 @ np.kron(sigmaz_basis(idx1,i),np.kron(ket_b[0],sigmaz_basis(idx2,j))))
            q = q + 0.5*(np.kron(np.conj(sigmaz_basis(idx1,i)).T,np.kron(bra_b[0],np.conj(sigmaz_basis(idx2,j)).T)) @ dm 
                 @ np.kron(sigmaz_basis(idx1,i),np.kron(ket_b[1],sigmaz_basis(idx2,j))))
            r = r + 0.5*(np.kron(np.conj(sigmaz_basis(idx1,i)).T,np.kron(bra_b[1],np.conj(sigmaz_basis(idx2,j)).T)) @ dm 
                 @ np.kron(sigmaz_basis(idx1,i),np.kron(ket_b[0],sigmaz_basis(idx2,j))))
            s = s + 0.5*(np.kron(np.conj(sigmaz_basis(idx1,i)).T,np.kron(bra_b[1],np.conj(sigmaz_basis(idx2,j)).T)) @ dm 
                 @ np.kron(sigmaz_basis(idx1,i),np.kron(ket_b[1],sigmaz_basis(idx2,j))))
    return np.array([[p,q],[r,s]]).reshape(2,2)

## Entanglement Entropy

In [9]:
def Entropy(rdm_c):
    lmbda = np.linalg.eigvals(rdm_c)
    S = 0
    for i in range(len(lmbda)):
        k = lmbda[i]
        if np.round(k,4) == 0 or np.round(k,4) == -0:
            S = S - 0
        else:
            S = S - k*np.log2(k)  
    return S

In [10]:
def biorthogonal_entropy(H,idx1,idx2):
    S_c = []
    for i in range(len(H)):
        ket_psi, ket_phi = eigvectors(H[i])
        N = np.sqrt(np.conj(ket_phi).T @ ket_psi)
        dm = (ket_psi/N) @ (np.conj(ket_phi).T/N)
        rdm_c = bi_rho_c(dm,idx1,idx2)
        #print(rdm_c)
        #print(N)
        S_c.append(Entropy(rdm_c))
    return(np.round(S_c,5))              #returns S_c rounded to 5 decimal places

## Time evolution

In [11]:
def plot_entropy_t(n,idx,idx1,idx2,hx,hz,d):
    
    #Intially at t=0 let psi be all spin up states
    ket_0 = np.array([0,1]).reshape(2,1)
    ket_1 = np.array([1,0]).reshape(2,1)
    psi_t0 = 1
    p=n
    while p>0:
        psi_t0 = np.kron(psi_t0,ket_1)
        p-=1
    H = Hamiltonian(n,idx,hx,hz,d)
    H_dag = []
    for i in range(len(H)):
        Hdagger = np.conj(H[i]).T
        H_dag.append(Hdagger)
        
    ket_psi_gs = []
    bra_phi_gs = []
    for j in range(len(H)):
        psi,phi = eigvectors(H[j])
        ket_psi_gs.append(psi)
        bra_phi_gs.append(np.conj(phi).T)
        
    S_gs = []    
    for k in range(len(H)):
        t = np.linspace(0,100,100,endpoint=True)
        S = []
        for i in range(len(t)):
            ket_psi = scipy.linalg.expm(-1j*H[k]*t[i])@psi_t0
            bra_psi = (np.conj(psi_t0).T)@scipy.linalg.expm(1j*H_dag[k]*t[i])
            N = np.sqrt(bra_psi @ ket_psi)
            rho = (ket_psi/N) @ (bra_psi/N)
            rdm_c = bi_rho_c(rho,idx1,idx2)
            S.append(np.round(Entropy(rdm_c),5))
            #S_ground = biorthogonal_entropy(H,idx1,idx2)
            #S_gs.append(S_ground)
            #print(S_ground)
        #print(S_gs)    
        plt.plot(t,S,color='b')
        #plt.plot(t,S_gs,color='g')
        plt.title('Entropy dynamics : (n,hz) = ({},{})'.format(n,hz[k]))
        plt.xlabel('time')
        plt.ylabel('Entanglement entropy')
        plt.ylim(0,2)
        plt.show()